# Search STAC API
This notebook uses the **PySTAC Client** (```pystac-client```, https://pystac-client.readthedocs.io/en/latest/) Python package to find suitable Sentinel-2 MSIL2A data by using a query with a number of defined paramteres. 

The STAC API for AWS is available on the endpoint (https://stac-api-dev.terradue.com/).

# Set-up

In [1]:
import pystac
from pystac_client import Client

from shapely.geometry import box, mapping, Polygon
import shapely.wkt

import os
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import xarray as xr
import stackstac

In [2]:
# Access to Catalog 
URL = 'https://stac-api-dev.terradue.com/'

headers = []

cat = Client.open(URL, headers=headers, ignore_conformance=True)
cat

/workspace/.conda/envs/env_stac_access/lib/python3.8/site-packages/pystac_client/client.py:177: FutureWarning: The `ignore_conformance` option is deprecated and will be removed in the next major release. Instead use `set_conforms_to` or `add_conforms_to` to control behavior.
  warnings.warn(


<Client id=stac-fastapi>

## Define params


In [9]:
# Collection 
collections=["ai-extensions-svv-dataset-labels"]

# Start and End dates
start_date_str = '2023-06-01'; start_date = datetime.fromisoformat(start_date_str)
stop_date_str = '2023-06-15'; stop_date = datetime.fromisoformat(stop_date_str)

# BBOX or AOI
# bbox = [-74.9211627, 17.818496, -72.2759771, 19.654895]
aoi='POLYGON((-120.90 37.55,-120.32 37.55,-120.32 37.10,-120.90 37.10,-120.90 37.55))'; bbox=shapely.wkt.loads(aoi).bounds 

# Other metadata
cloud_cover = 30

# Define EPSG code
epsg = 'EPSG:4326'

## Query the Catalog

In [10]:
# Query by AOI, start and end date and other params 
query = cat.search(
    collections=collections, 
    # datetime = (start_date, stop_date),
    # # sortby="properties.datetime",
    bbox=bbox, 
    # query = {
    #     'eo:cloud_cover': {'lt': 5}, # cloud_cover less than 15 percent by default
    #     # 'sentinel:product_id': {'eq': product_id}
    # }
)

In [11]:
# items_stack = query.get_all_items()
items = query.item_collection()

print(f'There are {len(items)} items in the collection')
display(items)

There are 4 items in the collection


In [6]:
bbox_list = []
for item in items: 
    print(item.id, item.properties['datetime'])
    bbox_list.append(item.bbox)
bbox_list

label_S2B_10SFH_20230613_0_L2A 2023-08-02T14:42:10.815029Z
label_S2B_10SFG_20230613_0_L2A 2023-08-02T14:41:34.319626Z
label_S2A_10SFH_20230618_0_L2A 2023-08-02T14:40:59.804516Z
label_S2A_11SKB_20230618_0_L2A 2023-08-02T14:40:22.215059Z
label_S2A_10SGG_20230618_0_L2A 2023-08-02T14:40:04.143647Z
label_S2A_10SFG_20230618_0_L2A 2023-08-02T14:39:34.232304Z


[[-121.86327090523541,
  37.835311345175796,
  -120.58327961102275,
  38.84330548198025],
 [-121.87680832296513,
  36.93451998232696,
  -120.6130695018397,
  37.94207619848921],
 [-121.86327090523541,
  37.835311345175796,
  -120.58327961102275,
  38.84330548198025],
 [-120.41196070892491,
  36.989424078854626,
  -120.06565988023824,
  37.90763021375156],
 [-120.75463878511704,
  36.93063805399626,
  -120.06532070709298,
  37.92558132143914],
 [-121.87680832296513,
  36.93451998232696,
  -120.6130695018397,
  37.94207619848921]]

In [8]:
# Show with folium
import folium

# Create a folium map centered on a specific location (e.g., California)
map_center = [37.5, -119.0]
mymap = folium.Map(location=map_center, zoom_start=7)

# Create separate FeatureGroups for each bounding box layer
for i, bbox in enumerate(bbox_list):
    feature_group = folium.FeatureGroup(name=f'BBOX {i+1}')
    mymap.add_child(feature_group)
    
    # Extract the coordinates
    min_lon, min_lat, max_lon, max_lat = bbox
    
    # Calculate the center of the bounding box
    center_lat = (min_lat + max_lat) / 2
    center_lon = (min_lon + max_lon) / 2
    
    # Create a rectangle using the bounding box coordinates
    rect = folium.Rectangle(
        bounds=[(min_lat, min_lon), (max_lat, max_lon)],
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3,
        tooltip=f'BBOX {i+1}'
    )
    
    # Add the rectangle to the feature group
    rect.add_to(feature_group)

# Add LayerControl to the map
folium.LayerControl().add_to(mymap)
mymap

In [ ]:
# Display properties of one item 
items[0].properties

**Note**: If the ```epsg``` and ```resolution``` are not defined in all Items/Assets, they must be explicitly defined in the ```stackstac.stack()``` call. The ```resolution``` refers to the output resolution and must be set in the same unit as the ```epsg``` field ([stackstac documentation](https://stackstac.readthedocs.io/en/latest/api/main/stackstac.stack.html)).

In [ ]:
items[0]

In [ ]:
# print id of the first item
index = 0
item_id = items[index].id 
print(f'id: {item_id}')
date = items[index].datetime
print(f'date: {date}')

# print available bands
print(f'bands: {list(items[index].assets.keys())}')

In [ ]:
import os
from osgeo import gdal, ogr, osr
import pyproj
import numpy as np
import pandas as pd
import json

gdal.UseExceptions()

In [ ]:
os.environ['PROJ_DATA']='/workspace/.conda/envs/env_labels/share/proj' 

In [ ]:
import pystac
from pystac import Link, Asset
from pystac.extensions.label import LabelExtension
from pystac.extensions.label import LabelType
from pystac.extensions.label import LabelClasses
from pystac.extensions.label import LabelStatistics
from pystac.extensions.version import ItemVersionExtension

In [ ]:
# Set dataframe to None 
df = None

In [ ]:
# Read GeoJSON file and extract point coordinates
def read_geojson_coordinates(geojson_file):
    with open(geojson_file, 'r') as file:
        geojson_data = json.load(file)
    #for f in geojson_data['features'][:10]: print(f)
    
    points = []
    luc = []
    for feature in geojson_data['features']:
        if feature['geometry']['type'] == 'Point':
            # Add lon and lat
            lon, lat, _ = feature['geometry']['coordinates']
            points.append((lon, lat))
            
            # Add classification 
            luc.append(feature['properties']['class'])
    return points, luc

In [ ]:
# Function to transform unprojected coordinates to projected coordinates
def transform_coordinates(coordinates, epsg_s, epsg_t):
    source_crs = pyproj.CRS(f'EPSG:{epsg_s}') 
    target_crs = pyproj.CRS(f'EPSG:{epsg_t}')  
    transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
    transformed_coords = [transformer.transform(lon, lat) for lon, lat in coordinates]
    
    transformed_coords_int = [[int(tc[0]), int(tc[1])] for tc in transformed_coords]
    return transformed_coords_int

In [ ]:
# Function to extract pixel values from a GeoTIFF at given coordinates
def extract_pixel_values(b_g, transformed_coords):
    gt = b_g.GetGeoTransform()
    b_rst = b_g.GetRasterBand(1)
    
    values = []

    for lon, lat in transformed_coords:
        px = int((lon - gt[0]) / gt[1])  # Convert longitude to pixel x
        py = int((lat - gt[3]) / gt[5])  # Convert latitude to pixel y

        value = b_rst.ReadAsArray(px, py, 1, 1)[0][0]
        values.append(value)
    
    # Empty raster 
    b_rst = None
    
    return values

## Read Label STAC Item

In [ ]:
# Define name of the ML item
item_label_fname = "item-label-train.json"

In [ ]:
item = pystac.read_file(item_label_fname)
display(item.properties)

## Load S2 scene

In [ ]:
s2_href = [l.target for l in item.links if l.rel == 'source'][0]
print('href of the S2 scene:', s2_href)

In [ ]:
# Read STAC Item
s2item = pystac.read_file(s2_href)
display(s2item.properties)

In [ ]:
epsg_t = s2item.properties['proj:epsg']
print(f'- Target EPSG:{epsg_t}')
print(f'- Available bands: {list(s2item.assets.keys())}')

## Load geojson points

In [ ]:
geojson_href = item.assets['labels'].href
print('href of the geojson file:', geojson_href)

In [ ]:
# Open asset (geojson format) and read all the coordinates within 
coordinates, luc = read_geojson_coordinates(geojson_href)
coordinates[:5], luc[:5]

In [ ]:
# Transofrm coordinates
epsg_s = '4326'
transformed_coords = transform_coordinates(coordinates, epsg_s, epsg_t)
transformed_coords[:5]

## Extract values of selected band(s) for each point in the geojson file


In [ ]:
# Define dictionary of other Common Band Names, for those that are missing in the metadata
other_cbn = {
    'B05': 'rededge70', 
    'B06': 'rededge74', 
    'B07': 'rededge78', 
    'B8A': 'nir08', 
    'B09': 'nir09'
}

In [ ]:
for band in list(s2item.assets.keys()): 
    b_metadata = s2item.assets[band].to_dict()
    if 'eo:bands' in b_metadata.keys() and len(b_metadata['eo:bands']) == 1:
        if 'common_name' in b_metadata['eo:bands'][0].keys(): 
            cbn = b_metadata['eo:bands'][0]['common_name']
        else: 
            # cbn does not exist in metadata - use dictionary of other_cbn
            cbn = other_cbn[b_metadata['eo:bands'][0]['name']]
    else: 
        print(f'{band} is not eo band, skipping.')
        continue

    if (df is not None) and (cbn in df.columns): 
        print(f'Band {cbn} exists already in the dataframe, skipping.')
        continue
    
    print('Band:', band)
    print(f'- Common Band Name: {cbn}')
    print(f'- Res: {b_metadata["gsd"]}m')
    print(f'- Center Wavelenght: {b_metadata["eo:bands"][0]["center_wavelength"]}')

    # Extract band
    b_href = s2item.assets[band].href
    print('- href:', b_href)

    # Get gdal object
    b_g = gdal.Open(b_href)
    
    # Extract pixel values
    pixel_values = extract_pixel_values(b_g, transformed_coords)
    # for (lon0, lat0), (lon1, lat1), value, lu in zip(coordinates[:5], transformed_coords[:5], pixel_values[:5], luc[:5]):
    #     print(f"Coords (Unprj): {np.round(lon0,3)}, {np.round(lat0,3)} - Coords (Prj): {lon1}, {lat1} - Pixel Value: {value} - LUC: {lu}")
    
    # Empty b_g
    b_g = None
    
    # Make or Append to Pandas dataframe
    data = {
        'long': [x[0] for x in transformed_coords], 
        'lat': [x[1] for x in transformed_coords], 
        'LUC': luc,
        cbn: pixel_values,
    }

    if df is None: 
        print('Creating Dataframe')
        # Create a DataFrame from the dictionary
        df = pd.DataFrame(data)
        df.index.name = 'Index'

    else: 
        print('Adding to existing Dataframe')

        # Create temp dataframe
        df2 = pd.DataFrame(data)
        df2.index.name = 'Index'

        # Assert the two dataframes have the same long and lat values
        assert df['long'].isin(df2['long']).value_counts().values[0] == len(pixel_values)
        assert df['lat'].isin(df2['lat']).value_counts().values[0] == len(pixel_values)

        # Merge temp dataframe with original dataframe, based on matching columns
        df = pd.merge(df, df2, on=['Index', 'long', 'lat', 'LUC'])   
        # Empty memory
        df2 = None

    display(df)
    print()

print('\n--- Complete Dataframe with all Sentinel-2 bands ---')
display(df)

## Add NDVI and NDWI bands

In [ ]:
# Add NDVI
assert 'nir' in df.columns and 'red' in df.columns
df['ndvi'] = ((df['nir'].astype(int) - df['red'].astype(int)) / (df['nir'].astype(int) + df['red'].astype(int)) * 10000).astype(int)
df

In [ ]:
# Add NDWI
# Formula for S2: NDWI = (NIR - MIR) / (NIR + MIR) using Sentinel-2 Band 8 (NIR) and Band 12 (MIR=SWIR for Sentinel2)
assert 'nir' in df.columns and 'swir16' in df.columns and 'swir22' in df.columns
df['ndwi1'] = ((df['nir'].astype(int) - df['swir16'].astype(int)) / (df['nir'].astype(int) + df['swir16'].astype(int)) * 10000).astype(int)
df['ndwi2'] = ((df['nir'].astype(int) - df['swir22'].astype(int)) / (df['nir'].astype(int) + df['swir22'].astype(int)) * 10000).astype(int)
df

## Add Water Label

In [ ]:
# Add Water label from the LUC value (LUC=6 is water)
df.loc[df['LUC'] == 6, 'water'] = 1
df.loc[df['LUC'] != 6, 'water'] = 0
df

## Show some statistics

In [ ]:
df.describe().T

## Export Dataframe to CSV

In [ ]:
# Export dataframe 
df.to_csv('df_extractedpixels.csv')

In [ ]:
print('END')